# Project Title
### Data Engineering Capstone Project

#### Project Summary
Immigration data and city temperature data provided by Udacity is going to be used to create a star schema that allow users to see if there is a relation between City Temperature and immigration requests to this City. 

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import re
from pyspark.sql.functions import udf

### Step 1: Scope the Project and Gather Data

#### Scope 
The scope of the project is to provide an analytical schema for users using provided immigration dataset and temperature dataset so user can use the generated model to find answer for questions like is there a relationship between city temperature and number of immigrant? Do people from countries with warmer or cold climate immigrate to the US in large numbers?

The country dimension table is made up of data from the global land temperatures by city and the immigration datasets. The combination of these two datasets allows analysts to study correlations between global land temperatures and immigration patterns to the US and get insights into migration patterns into the US based on the temperature, demographics as well as overall population of states. We could also ask another questions such as, do populous states attract more visitors on a monthly basis?

#### Describe and Gather Data 
##### I94 Immigration Data:
This dataset comes from the US National Tourism and Trade Office. it is provided as SAS files by Udacity.

###### main fields in the data set are
- i94yr   : 4 digit year
- i94mon  : numeric month
- i94cit  : code of origin city
- i94port : code of destination city
- arrdate : arrival date
- i94mode : travel code
- depdate : departure date
- i94visa : Visa codes collapsed into three categories Business, Pleasure and Student
- i94bir  : Age of Respondent in Years

##### World Temperature Data: 
This dataset came from Kaggle and provided as CSV file by Udacity.
this dataset has the follwoing fields AverageTemperature, City, Country, Latitude and Longitude


### Immigration Data

In [2]:
#first get all SAS data files and store them in list
#os.path.join(os.getcwd(), 'data/18-83510-I94-Data-2016')
#os.listdir('../../data/18-83510-I94-Data-2016')
files_list = [os.path.join('../../', 'data/18-83510-I94-Data-2016', f) for f in os.listdir('../../data/18-83510-I94-Data-2016')]
files_list

['../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_sep16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_nov16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_mar16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_aug16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_jan16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_oct16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_jul16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_feb16_sub.sas7bdat',
 '../../data/18-83510-I94-Data-2016/i94_dec16_sub.sas7bdat']

In [ ]:
# Read in the data here
# get one file of the file list and work on it
fname = files_list[0] 
#fname #'../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")

In [5]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


### Temperature Data

In [11]:
Temperature_File = '../../data2/GlobalLandTemperaturesByCity.csv'
df_Temp = pd.read_csv(Temperature_File)

In [4]:
df_Temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [4]:
spark

In [11]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 


## Immigration Data

In [14]:
# see all columns in the immigration df
df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [15]:
# get unique i94port codes 
df['i94port'].unique()

array(['XXX', 'ATL', 'WAS', 'NYC', 'TOR', 'BOS', 'HOU', 'MIA', 'CHI',
       'LOS', 'CLT', 'DEN', 'DAL', 'DET', 'NEW', 'FTL', 'LVG', 'ORL',
       'NOL', 'PIT', 'SFR', 'SPM', 'POO', 'PHI', 'SEA', 'SLC', 'TAM',
       'HAM', 'NAS', 'VCV', 'MAA', 'AUS', 'HHW', 'OGG', 'PHO', 'SDP',
       'SFB', 'EDA', 'MON', 'CLG', 'DUB', 'FMY', 'YGF', 'SAJ', 'CIN',
       'BAL', 'RDU', 'WPB', 'STT', 'OAK', 'NSV', 'SNA', 'OTT', 'X96',
       '5KE', 'CLE', 'HAR', 'PSP', 'CHR', 'HAL', 'SAA', 'KOA', 'SHA',
       'WIN', 'BGM', 'NCA', 'OPF', 'SAI', 'JFA', 'AGA', 'ONT', 'CLM',
       'STL', 'W55', 'CHS', 'SNJ', 'SRQ', 'ANC', 'LNB', 'LIH', 'MIL',
       'INP', 'KAN', 'ROC', 'SAC', 'BRO', 'LAR', 'RNO', 'SGR', 'ELP',
       'MCA', 'MDT', 'SPE', 'FPR', 'SYR', 'ICT', 'MLB', 'ADS', 'TUC',
       'DLR', 'CAE', 'CHA', 'HSV', 'WIL', 'HPN', 'HEF', 'BRG', 'BED',
       'DAB', 'JAC', 'FRB', 'SWF', 'KEY', 'PTK', 'MWH', 'X44', 'MYR',
       'APF', 'ATW', 'PVD', 'BUF', 'PIE', 'MHT', 'BDL', 'NYL', 'VNY',
       '5T6', 'LEX',

In [16]:
# get unique values in i94bir column
df['i94bir'].unique()

array([  37.,   25.,   55.,   28.,    4.,   57.,   63.,   46.,   48.,
         52.,   33.,   58.,   56.,   62.,   49.,   43.,   53.,   74.,
         65.,   35.,   32.,   38.,   68.,   61.,   41.,   45.,   54.,
         29.,   42.,   34.,   47.,   64.,   27.,   59.,   60.,   66.,
         51.,   22.,   39.,   20.,   50.,   44.,   40.,   31.,   23.,
         36.,    2.,    0.,   70.,   26.,   30.,   16.,   14.,   21.,
         24.,    1.,   77.,   73.,   71.,    6.,   72.,    5.,   76.,
         69.,   67.,    3.,   10.,   18.,   19.,   11.,   17.,    9.,
          8.,   12.,   75.,    7.,   13.,   15.,   82.,   84.,   78.,
         81.,   87.,   79.,   80.,   83.,   91.,   85.,   86.,   88.,
         90.,   89.,   97.,   96.,   93.,   92.,  100.,   95.,   98.,
         94.,   99.,   nan,  109.,  108.,  107.,  101.,  105.,  102.,
        103.,   -3.,  114.,  110.,  111.])

In [17]:
# see maximum value of i94bir column 
max(df['i94bir'])

114.0

In [18]:
# see minimum value of i94bir column 
min(df['i94bir'])

-3.0

##### we can clean immigration data by the following
- drop rows for invalid destination cities (i94port column), we can get the vaild city code from description file (lines 303 to 893)
- drop rows with invalid 194bir like -3 and nan , the values should be between 1 and 115
- keep relevant columns only as we have 28 columns and we do not need all the columns

In [6]:
# get valid i94port codes and cities only and save it as dictionary of {code: list}
# i copied the codes from SAS Description file and save it as new file and removed invalid values so it is easy for me to read it

#with open('I94_SAS_Labels_Descriptions.SAS') as f:
#    valid_lines = f.readlines()[302:893]

with open('valid_ports_me.txt') as f:
    valid_lines = f.readlines()
    
re_obj = re.compile(r'\'(.*)\'.*\'(.*)\'')
valid_ports = {}
for line in valid_lines:
    match = re_obj.search(line)
    valid_ports[match.group(1)]=[match.group(2)]

#valid_ports

In [7]:
'''
now we can start to clean immigrtion data as we have valid i94port codes
this function take file path and do the following
1- read the file as spark dataframe
2- drop rows with invalid values in i94port column
3- drop rows with i94bir value not between 1 and 116 years
4- select the desired columns only from the whole 28 columns
'''
def get_Immigration_Spark_df_from_SAS_file(file_path):
    '''
    Input: File Path for SAS File to be loaded
    
    Output: Cleansed and ready to use Spark DataFrame with Immigration Data
    
    '''
    spark_df = spark.read.format('com.github.saurfang.sas.spark').load(files_list[0])
    spark_df = spark_df.filter(spark_df.i94port.isin(list(valid_ports.keys())))
    spark_df = spark_df.filter(spark_df.i94bir.between(1,116))
    spark_df = spark_df.select(['i94yr', 'i94mon', 'i94cit', 'i94port', 'i94mode', 'i94bir', 'arrdate', 'depdate', 'i94visa'])
    return spark_df

In [8]:
# test the function with one file
immigration_df = get_Immigration_Spark_df_from_SAS_file(files_list[0])

In [8]:
# display the first 5 rows in the generated df
immigration_df.show(5)

+------+------+------+-------+-------+------+-------+-------+-------+
| i94yr|i94mon|i94cit|i94port|i94mode|i94bir|arrdate|depdate|i94visa|
+------+------+------+-------+-------+------+-------+-------+-------+
|2016.0|   4.0| 254.0|    ATL|    1.0|  25.0|20551.0|   null|    3.0|
|2016.0|   4.0| 101.0|    WAS|    1.0|  55.0|20545.0|20691.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|    1.0|  28.0|20545.0|20567.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|    1.0|   4.0|20545.0|20567.0|    2.0|
|2016.0|   4.0| 101.0|    NYC|    1.0|  57.0|20545.0|20555.0|    1.0|
+------+------+------+-------+-------+------+-------+-------+-------+
only showing top 5 rows



#### Temperature Data Exploration

In [8]:
# see the first 5 rows in df
df_Temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [9]:
df_Temp.columns

Index(['dt', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude'],
      dtype='object')

In [10]:
df_Temp.shape

(8599212, 7)

In [11]:
df_Temp.drop_duplicates(['City', 'Country']).shape

(3490, 7)

In [12]:
df_Temp['City'].unique()

array(['Århus', 'Çorlu', 'Çorum', ..., 'Zurich', 'Zuwarah', 'Zwolle'], dtype=object)

In [14]:
df_Temp.describe(include='all')

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
count,8599212,8.235082e+06,8.235082e+06,8599212,8599212,8599212,8599212
unique,3239,NaN,NaN,3448,159,73,1227
top,1959-08-01,NaN,NaN,Springfield,India,36.17N,139.23E
freq,3510,NaN,NaN,9545,1014906,425455,129600
mean,NaN,1.672743e+01,1.028575e+00,NaN,NaN,NaN,NaN
std,NaN,1.035344e+01,1.129733e+00,NaN,NaN,NaN,NaN
min,NaN,-4.270400e+01,3.400000e-02,NaN,NaN,NaN,NaN
25%,NaN,1.029900e+01,3.370000e-01,NaN,NaN,NaN,NaN
50%,NaN,1.883100e+01,5.910000e-01,NaN,NaN,NaN,NaN
75%,NaN,2.521000e+01,1.349000e+00,NaN,NaN,NaN,NaN


In [15]:
# display max and min values in AverageTemperature column to see if there is any outliers
print(max(df_Temp['AverageTemperature']))
print(min(df_Temp['AverageTemperature']))

39.650999999999996
-42.70399999999999


In [8]:
''' 
it seems that the average temerpature is valid but we need to remove duplicate rows and rows with Nan and we need to add i94port code as a 
new column to temperature data set so we can join it with immigration data set we achive this by creation a user defined function that take the 
city name and return i94code from valid codes dictionary we built above
'''

@udf()
def get_i94port_Code_By_City(city_name):
    '''
    Input: City name    
    Output: the corresponding i94port value if founded    
    '''
    
    for k in valid_ports:
        if city_name.upper() in valid_ports[k][0].upper():
            return k

In [9]:
'''
now we will build the function that take Temperature data file path and read it as spark data frame
then clean it and return to us a clean and ready to use Spark DataFrame
1- read CSV File as Spark DataFrame
2- drop duplicates rows by city and country
3- drop rows with AverageTemperature = NaN
4- drop rows with i94port = null
5- add new column i94port by lookup city name in valid ports dictionary
6- return the DataFrame
'''
def get_Temperature_spark_df_from_CSV_file(file_path):
    '''
        Input: file path that contains Temperature Data (csv file)
        Output: clean and ready to use Spark DataFrame with Temperature Data
    '''
    temperature_df = spark.read.format("csv").option("header", "true").load(file_path)
    temperature_df = temperature_df.filter(temperature_df.AverageTemperature != 'NaN')
    temperature_df = temperature_df.dropDuplicates(['City', 'Country'])
    temperature_df = temperature_df.withColumn("i94port", get_i94port_Code_By_City(temperature_df.City))
    temperature_df = temperature_df.filter(temperature_df.i94port != 'null')
    return temperature_df

In [12]:
# test the function and display the first 5 rows
temperature_df = get_Temperature_spark_df_from_CSV_file(Temperature_File)
temperature_df.show(5)

+----------+------------------+-----------------------------+--------+-------------+--------+---------+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|    City|      Country|Latitude|Longitude|i94port|
+----------+------------------+-----------------------------+--------+-------------+--------+---------+-------+
|1852-07-01|            15.488|                        1.395|   Perth|    Australia|  31.35S|  114.97E|    PER|
|1828-01-01|            -1.977|                        2.551| Seattle|United States|  47.42N|  121.97W|    SEA|
|1743-11-01|             2.767|                        1.905|Hamilton|       Canada|  42.59N|   80.73W|    HAM|
|1849-01-01| 7.399999999999999|                        2.699| Ontario|United States|  34.56N|  116.76W|    ONT|
|1821-11-01|             2.322|                        2.375| Spokane|United States|  47.42N|  117.24W|    SPO|
+----------+------------------+-----------------------------+--------+-------------+--------+---------+-

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
we will create a star schema with the two dimensions tables and on fact table because the star schema is widly used and easy
to understand by users

- Dimension Tables
    - Demographic_Dim: has the following columns and populated from the I94 immigration data set.
        - I94YR   : year
        - I94MON  : month
        - I94CIT  : origion city code
        - I94PORT : destionation city code
        - I94MODE : travel code
        - I94BIR  : Age of Respondent in Years between 1 and 116
        - ARRDATE : Arrival date
        - DEPDATE : Departure date
        - I94VISA : Visa type (Business/Pleasure/Student)

    - Temperature_Dim: has the following columns and populated from the temperature dataset.
        - AverageTemperature : Average Temperature
        - City               : City Name
        - Country            : Country Name 
        - Latitude           : Latitude
        - Longitude          : Longitude
        - I94PORT            : city code mapped from valid codes in SAS Description File

- Immigration_Fact: has the following columns populated from immigration data set and temperature dataset
    - year              : year (I94YR)
    - month             : month (I94MON)
    - origion_city      : origion city code (I94CIT)
    - destionation_city : destionation city code (I94PORT)
    - travel_code   	: travel code (I94MODE)
    - respondent_age	: Age of Respondent in Years between 1 and 116 (I94BIR)
    - arrival_date		: Arrival date (ARRDATE)
    - departure_date	: Departure date (DEPDATE)
    - visa_type			: Visa type (Business/Pleasure/Student) (I94VISA)
    - avgTemp 			: Average Temperature


#### 3.2 Mapping Out Data Pipelines
- load immigration sas files and clean it using function get_Immigration_Spark_df_from_SAS_file wich return clean and ready to use spark DataFrame
- write this spark dataframe as parquet file partitiond by i9port(destination city)

- load Temperature CSV file and clean it using function get_Temperature_spark_df_from_CSV_file wich return clean and ready to use spark DataFram
- write this spark dataframe as parquet file partitiond by i9port(destination city)

- create fact table by joining the above two DataFrames and write to parquet file partition by i94port(destination city)

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [13]:
# load immigration data
immigration_df = get_Immigration_Spark_df_from_SAS_file(files_list[0])
# wirite data to parquet file
immigration_df.write.mode("append").partitionBy("i94port").parquet("DWH/immigration_data.parquet")

In [14]:
# load temperature data
temperature_df = get_Temperature_spark_df_from_CSV_file(Temperature_File)
# wirite data to parquet file
temperature_df.write.mode("append").partitionBy("i94port").parquet("DWH/temperature_data.parquet")

In [15]:
# Create temporary views of the two data frames to create fact table 
immigration_df.createOrReplaceTempView("immi_view")
temperature_df.createOrReplaceTempView("temp_view")

In [16]:
# Create the fact table by joining the immigration and temperature views
fact_table = spark.sql("""
SELECT i.i94yr as year,
       i.i94mon as month,
       i.i94cit as orgin_City,
       i.i94port as destination_city,
       i.i94mode as travel_code,
       i.i94bir as respondent_age,
       i.arrdate as arrival_date,
       i.depdate as depature_date,
       i.i94visa as visa_type,
       t.AverageTemperature as avgTemp
FROM immi_view i
JOIN temp_view t ON (i.i94port = t.i94port)
""")

# Write fact table to parquet files partitioned by i94port
fact_table.write.mode("append").partitionBy("destination_city").parquet("DWH/fact.parquet")

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [17]:
# check if df has data or not
def count_quality_check(df):
    '''
    Input: Spark DataFrame
    Output: 0 for Success and -1 for Failed
    '''
    if df.count() == 0:
        print('Count Quality Check Failed, Empty DataFrame')
        return -1
    else:
        print('Count Quality Check Successed')
        return 0

# check integrity between immigration and temperature dataframes
def integrity_quality_check(df_immigration, df_temp):
    '''
    Input: two Spark DataFrames
    Output: 0 for Success and -1 for Failed
    '''
    if df_immigration.select("i94port").distinct().join(df_temp, df_immigration["i94port"] == df_temp["i94port"], "left_anti")\
    .count() == 0:
        print('Integrity Quality Check Failed')
        return -1
    else:
        print('Integrity Quality Check Successed')
        return 0

In [2]:
if count_quality_check(immigration_df)  == 0 and count_quality_check(temperature_df) == 0 \
and integrity_quality_check(immigration_df,temperature_df) == 0:
    print('Success')
else:
    print('Failed')

Success


#### 4.3 Data dictionary 

- Dimension Tables
    - Demographic_Dim: has the following columns and populated from the I94 immigration data set.
        - I94YR   : 4 digits year
        - I94MON  : numeric month
        - I94CIT  : origion city code
        - I94PORT : destionation city code
        - I94MODE : travel code
        - I94BIR  : Age of Respondent in Years between 1 and 116
        - ARRDATE : Arrival date
        - DEPDATE : Departure date
        - I94VISA : Visa type (Business/Pleasure/Student)

    - Temperature_Dim: has the following columns and populated from the temperature dataset.
        - AverageTemperature : Average Temperature
        - City               : City Name
        - Country            : Country Name 
        - Latitude           : Latitude
        - Longitude          : Longitude
        - I94PORT            : destionation city code mapped from valid codes in SAS Description File.

- Immigration_Fact: has the following columns populated by joining the two dim tables
    - year              : year (I94YR)
    - month				: month (I94MON)
    - origion_city 		: origion city code (I94CIT)
    - destionation_city : destionation city code (I94PORT)
    - travel_code   	: travel code (I94MODE)
    - respondent_age	: Age of Respondent in Years between 1 and 116 (I94BIR)
    - arrival_date		: Arrival date (ARRDATE)
    - departure_date	: Departure date (DEPDATE)
    - visa_type			: Visa type (Business/Pleasure/Student) (I94VISA)
    - avgTemp 			: Average Temperature


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.

  - For exploring the data i used panda libirary as it is easy to use, then for processing data i prefered to use Spark as it can handly many files with large amount of data and varies data formats. also spark sql used to perform standard SQL like operations on the data like joining the two dataframes to generate the fact table

  - We used star schema model not only it is the most common modeling paradigm but because it tends to be better for performance and it has a small number of tables and clear join paths, queries run faster than they do against an OLTP system. Small single-table queries, usually of dimension tables, are almost instantaneous.

* Propose how often the data should be updated and why.
since the data arrive monthly in files so we do not have the complexity of defining delta so we can load data every month when the data files arrive to our side and append to the previous data

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     - we can increase the work power of the spark cluster (maybe using autoscaling service here will be very helpful, when we got the files and start our batch job the cluster will increase and after that it will shrink) or we can change to Amazon Redshift DWH as it is scalable and can handle large amount of data
     
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - we can use a tool like Airflow to schedule the batch job to run daily and configure the success and fail scenario
    
 * The database needed to be accessed by 100+ people.
     - with this new settings we can go for Redshift Database as it is very scalable and can handle this number of requests. 